In [1]:
import random
import itertools
import pickle
import os
from abc import ABC, abstractmethod

In [2]:
class MovingCreature(ABC):
    
    
    @abstractmethod
    def __init__(self, x: int, y: int):
        pass
    
    
    @abstractmethod
    def move(self):
        pass

In [3]:
class Player(MovingCreature):
    
    
    def __init__(self, x, y):
        self.inventory = []
        self.exit_seen = False
        self.hp = 100
        self.x = x
        self.y = y
        
        
    def get_item(self, item):
        self.inventory.append(item)
        
        
    def move(self, direction):
        if direction == 'up':
            self.y -= 1
        elif direction == 'down':
            self.y += 1
        elif direction == 'left':
            self.x -= 1
        elif direction == 'right':
            self.x += 1
        else:
            assert False
           
        
    def check_item(self, item):
        return item in self.inventory
    
    
    def lose_hp(self, dmg):
        self.hp = self.hp - dmg
        if self.hp > 0:
            return True
        else:
            return False

In [4]:
class Bear(MovingCreature):
    
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.dmg = 50

        
    def move(self, direction):
        if direction == 'up':
            self.y -= 1
        elif direction == 'down':
            self.y += 1
        elif direction == 'left':
            self.x -= 1
        elif direction == 'right':
            self.x += 1
        else:
            assert False
    
    
    def hit(self, target):
        return target.lose_hp(self.dmg)

In [5]:
class Item(ABC):
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def __str__(self):
        pass

In [6]:
class Treasure(Item):
    
    def __init__(self):
        self.name = 'Treasure'
        self.value = 10
        
    def __str__(self):
        return self.name
    
    def buy_values(self):
        return self.value
    
    def sell_value(self):
        return self.value / 2

In [7]:
class Cell:
    
    def __init__(self, x, y, walls):
        
        self.x = x
        self.y = y
        self.walls = walls

        self.seen = False
      
    
    def direction(self, cell_to):
        if abs(self.x - cell_to.x) + abs(self.y - cell_to.y) == 1:
            if cell_to.y < self.y:
                return 'up'
            elif cell_to.y > self.y:
                return 'down'
            elif cell_to.x < self.x:
                return 'left'
            elif cell_to.x > self.x:
                return 'right'
            else:
                return False
        else:
            return False
        
        
    def connect(self, cell_to):
        cell_to.walls.remove(cell_to.direction(self))
        self.walls.remove(self.direction(cell_to))
        
        
    def isolated(self):
        return len(self.walls) == 4
    
    
    def explored(self):
        self.seen = True

In [121]:
class Labyrinth:
    
    def __init__(self, size):
        self.size = size
        self.size_sq = size ** 2
        self.cells = [[Cell(x, y, set(['up', 'down', 'right', 'left'])) for y in range(self.size)] 
                      for x in range(self.size)]
        
        self.symbols = {(): '┼', ('left'): '├', ('down'): '┴', ('up'): '┬',
                        ('up', 'left'): '┌', ('down', 'left'): '└', ('up', 'down'): '─',
                        ('up', 'down', 'left'): '╶', ('right'): '┤', ('right', 'left'): '│',
                        ('down', 'right'): '┘', ('up', 'right'): '┐', ('up', 'right', 'left'): '╷', 
                        ('down', 'right', 'left'): '╵', ('up', 'down', 'right'): '╴'}
      
    
    def neighbors(self, cell):
        x = cell.x
        y = cell.y
        res = []
        
        for new_x, new_y in [(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)]:
            if 0 <= new_x < self.size and 0 <= new_y < self.size:
                neighbor = self.cells[new_x][new_y]
                res.append(neighbor)
        
        return res
    
    
    def get_random_cell(self):
        cell = random.choice(self.cells)
        cell = random.choice(cell)
        
        return cell
      
        
    def create_maze(self):
        cell_stack = []
        cell = self.get_random_cell()
        n_visited_cells = 1

        while n_visited_cells < self.size_sq:
            neighbors = [c for c in self.neighbors(cell) if c.isolated()]
            if len(neighbors):
                neighbor = random.choice(neighbors)
                cell.connect(neighbor)
                cell_stack.append(cell)
                cell = neighbor
                n_visited_cells += 1
            else:
                cell = cell_stack.pop()
                
                
    def create_river(self):
        self.river = []
        
        cell = self.get_random_cell()
        self.river.append((cell.x, cell.y))
        dir_from = None
        dir_to = None
        
        walls = set(['up', 'down', 'right', 'left'])
        
        reverse = {'right':'left',
                   'left':'right',
                   'up':'down',
                   'down':'up'}
        
        while len(self.river) < self.size:
            
            try:
                directions = list(walls.difference(cell.walls))
                if dir_from in directions:
                    directions.remove(dir_from)

                if not len(directions):
                    break

                dir_to = random.choice(directions)
                dir_from = reverse[dir_to]


                if dir_to == 'right':
                    x = self.river[-1][0] + 1
                    y = self.river[-1][1] + 0
                    cell = self.cells[x][y]
                elif dir_to == 'left':
                    x = self.river[-1][0] - 1
                    y = self.river[-1][1] + 0
                    cell = self.cells[x][y]
                elif dir_to == 'up':
                    x = self.river[-1][0] + 0
                    y = self.river[-1][1] - 1
                    cell = self.cells[x][y]
                elif dir_to == 'down':
                    x = self.river[-1][0] + 0
                    y = self.river[-1][1] + 1
                    cell = self.cells[x][y]

                if (cell.x, cell.y) not in self.river:
                    self.river.append((cell.x, cell.y))
                else:
                    break

                if self.size * random.random() < 0.1:
                    break
            except IndexError:
                break
            
        
    def create_objects(self):
        idxs = [i for i in itertools.product(range(self.size), repeat=2) if i not in self.river]
        obj_pos = random.sample(idxs, 6)
        
        self.treasure = obj_pos[0]
        self.treasure_item = Treasure()
        self.wormholes = obj_pos[1:]
        
        
    def goto_next_wormhole(self, x, y):
        idx = (self.wormholes.index((x, y)) + 1) % 5
        
        return self.wormholes[idx]
        
        
    def create_monolith(self):
        monolith_idxs = range(-1, self.size+1)
        self.monolith = []
        
        self.monolith += [(-1, i) for i in monolith_idxs]
        self.monolith += [(i, -1) for i in monolith_idxs]
        self.monolith += [(self.size, i) for i in monolith_idxs]
        self.monolith += [(i, self.size) for i in monolith_idxs]
        
        self.monolith = list(set(self.monolith))
        edges = [(-1, -1), (-1, self.size), (self.size, self.size), (self.size, -1)]
        self.exit = random.choice([i for i in self.monolith if i not in edges])
        
    
    def create_map(self):
        self.map = [[0 for i in range(self.size + 2)] 
                    for j in range(self.size + 2)]
        
        keys = list(self.symbols.keys())

        for i in range(self.size + 2):
            for j in range(self.size + 2):
                if (i-1, j-1) in self.monolith:
                    if (i-1, j-1) == self.exit:
                        self.map[i][j] = ' '
                    else:
                        self.map[i][j] = '#'
                else:
                    if (i-1, j-1) in self.wormholes:
                        self.map[i][j] = '@'
                    elif (i-1, j-1) == self.treasure:
                        self.map[i][j] = '$'
                    elif (i-1, j-1) == self.river[0]:
                        self.map[i][j] = '*'
                    elif (i-1, j-1) in self.river:
                        self.map[i][j] = '≈'
                    else:
                        for k in range(len(self.symbols)):
                            if set(self.cells[i-1][j-1].walls) == set(keys[k]):
                                self.map[i][j] = self.symbols[keys[k]]

                            elif len(self.cells[i-1][j-1].walls) == 0:
                                self.map[i][j] = self.symbols[()]

                            elif list(self.cells[i-1][j-1].walls)[0] == keys[k]:
                                self.map[i][j] = self.symbols[keys[k]]
            
            
    def create_labyrinth(self):
        self.create_maze()
        self.create_river()
        self.create_objects()
        self.create_monolith()
        self.create_map()
        
        return self
    
    
class LabyrinthFabric:
    
    def produce(self, size):
        self.labyrinth = Labyrinth(size)
        lab = self.labyrinth.create_labyrinth()
        
        return lab

In [113]:
import random
import itertools
import pickle
import os

from labyrinth.labyrinth import *
from creatures.player import Player
from creatures.bear import Bear
from objects.implementation import Treasure


class LabyrinthGame:
    
    def __init__(self, labyrinth):
        self.labyrinth = labyrinth
        
        self.commands = {'quit': self.quit,
                         'skip': self.skip,
                         'pos': self.pos,
                         'clear': self.clear,
                         'inventory': self.inventory,
                         'exit': self.exit,
                         'map': self.print_seen_part,
                         'full_map': self.print_labyrinth,
                         'save': self.save,
                         'me': self.show_me,
                         'help': self.allcommands,
                         'go': self.move,
                         'hp': self.get_hp,
                        }
        
        self.move_commands = {'go up': ('up', 0, -1), 
                              'go down': ('down', 0, 1),
                              'go left': ('left', -1, 0),
                              'go right': ('right', 1, 0),
                             }
    
    
    def allcommands(self, command):
        '''Print all commands'''
        self.clear(command)
        
        for name, func in self.commands.items():
            print(f"'{name}': {func.__doc__}")
            
        return True
    
    
    def quit(self, command):
        '''Quit the game'''
        print('The game is closed')
        
        return False
    
    
    def skip(self, command):
        '''Skip your turn, fall into warmhole'''
        if (self.player.x, self.player.y) in self.labyrinth.river:
            idx = self.labyrinth.river.index((self.player.x, self.player.y))
            if idx + 2 < len(self.labyrinth.river):
                self.player.x, self.player.y = self.labyrinth.river[idx + 2]
                print('Skip move on the river, moved 2 steps down the river')
            else:
                self.player.x, self.player.y = self.labyrinth.river[-1]
                print('Skip move on the river, moved to estuary!')
                
        elif (self.player.x, self.player.y) in self.labyrinth.wormholes:
            print('You falled into wormhole!')
            self.player.x, self.player.y = self.labyrinth.goto_next_wormhole(self.player.x, self.player.y)
            self.labyrinth.cells[self.player.x][self.player.y].explored()
        else:
            print('You skipped your turn')
        
        return True
    
    
    def pos(self, command):
        '''Get your current position'''
        print(self.player.x, self.player.y)
        
        return None
    
    
    def clear(self, command):
        '''Clear output in console'''
        os.system('clear')
        
        return None
     
    
    def exit(self, command):
        '''Position of exit, cheating'''
        print(self.labyrinth.exit)
        
        return None
    
    
    def inventory(self, command):
        '''Open your inventory'''
        if self.player.inventory:
            for item in self.player.inventory:
                print(item)
        else:
            print('Your inventory is empty')
        
        return None
    
    
    def save(self, command):
        '''Save and quit the game'''
        filename = command.split()[1]
        
        with open(os.path.join('saves', filename) + '.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
        
        print(f'Game has been saved as {filename}')
        
        return False
    
    
    def move(self, command):
        '''Move into one direction of: "up", "down", "right", "left"'''
        direction, difx, dify = self.move_commands[command]
        
        if (self.player.x + difx, self.player.y + dify) in self.labyrinth.river:
            
            for x, y in self.labyrinth.river:
                self.labyrinth.cells[x][y].explored()
                
            if (self.player.x, self.player.y) not in self.labyrinth.river:
                idx = self.labyrinth.river.index((self.player.x + difx, self.player.y + dify))
                if idx + 2 < len(self.labyrinth.river):
                    self.player.x, self.player.y = self.labyrinth.river[idx + 2]
                    print('You reach the river, moved 2 steps down the river!')
                    return True
                
                else:
                    self.player.x, self.player.y = self.labyrinth.river[-1]
                    print('You reach the river, moved to estuary!')
                    return True
            
        
        if (self.player.x + difx, self.player.y + dify) in self.labyrinth.monolith:
            if (self.player.x + difx, self.player.y + dify) == self.labyrinth.exit:
                if self.player.check_item(self.labyrinth.treasure_item):
                    print('You found the treasure and left the labyrinth!')
                    self.__won = True
                else:
                    self.player.exit_seen = True
                    print("You cannot leave the labyrinth until you find the treasure")
            else:
                print("step impossible, monolith")
                return None
            
        elif direction in self.labyrinth.cells[self.player.x][self.player.y].walls:
            print("step impossible, wall")
            return None
            
        else:
            self.player.move(direction)
            self.labyrinth.cells[self.player.x][self.player.y].explored()
                
            if (self.player.x, self.player.y) == self.labyrinth.treasure:
                self.player.get_item(self.labyrinth.treasure_item)
                self.labyrinth.treasure = None
                print("step executed, treasure")
                
            elif (self.player.x, self.player.y) in self.labyrinth.wormholes:
                print("step executed, wormhole")
            elif (self.player.x, self.player.y) in self.labyrinth.river:
                print("step executed, river")   
            else:
                print("step executed")
        
        return True
            

    def print_labyrinth(self, command):
        '''Print all labyrinth, cheating'''
        self.clear(command)
        for i in range(len(self.labyrinth.map)):
            for j in range(len(self.labyrinth.map)):
                if (i-1, j-1) == (self.player.y, self.player.x):
                    print('x', end='')
                elif (i-1, j-1) == (self.bear.y, self.bear.x):
                    print('B', end='')
                else:
                    print(self.labyrinth.map[j][i], end='')
            print()
        return None
    
    
    def print_seen_part(self, command):
        '''Print seen part of labyrinth'''
        self.clear(command)
        for i in range(len(self.labyrinth.map)):
            for j in range(len(self.labyrinth.map)):
                if self.labyrinth.map[j][i] == ' ' and not self.player.exit_seen:
                    print('#', end='')
                    continue
                try:
                    if self.labyrinth.cells[j-1][i-1].seen:
                        print(self.labyrinth.map[j][i], end='')
                    else:
                        print('#', end='')    
                except IndexError:
                    print(self.labyrinth.map[j][i], end='')
            
            print()
        return self.skip(command)
            
    
    def show_me(self, command):
        '''Print labyrinth with you as "x" mark and bear as "B" mark if his cell is seen'''
        self.clear(command)
        for i in range(len(self.labyrinth.map)):
            for j in range(len(self.labyrinth.map)):
                if self.labyrinth.map[j][i] == ' ' and not self.player.exit_seen:
                    print('#', end='')
                    continue
                if (i-1, j-1) == (self.player.y, self.player.x):
                    print('x', end='')
                    continue
                try:
                    if self.labyrinth.cells[j-1][i-1].seen:
                        if (i-1, j-1) == (self.bear.y, self.bear.x):
                            print('B', end='')
                        else:
                            print(self.labyrinth.map[j][i], end='')
                    else:
                        print('#', end='')    
                except IndexError:
                    print(self.labyrinth.map[j][i], end='')
            
            print()
        return self.skip(command)
    
    
    def bear_walking(self):
        '''Random walking without tracing'''
        
        walls = set(['up', 'down', 'right', 'left', 'skip'])
        directions = list(walls.difference(self.labyrinth.cells[self.bear.x][self.bear.y].walls))
        moves = []
        
        # To lower probability of skip
        for move in directions:
            if move == 'skip':
                moves.append(move)
            else:
                for _ in range(3):
                    moves.append(move)
        
        # Pick random action
        action = random.choice(moves)
        
        if action == 'skip':
            if (self.bear.x, self.bear.y) in self.labyrinth.river:
                idx = self.labyrinth.river.index((self.bear.x, self.bear.y))
                if idx + 2 < len(self.labyrinth.river):
                    self.bear.x, self.bear.y = self.labyrinth.river[idx + 2]
                else:
                    self.bear.x, self.bear.y = self.labyrinth.river[-1]
            
            elif (self.bear.x, self.bear.y) in self.labyrinth.wormholes:
                self.bear.x, self.bear.y = self.labyrinth.goto_next_wormhole(self.bear.x, self.bear.y)
        else:
            command = 'go ' + action
            direction, difx, dify = self.move_commands[command]
            
            if (self.bear.x + difx, self.bear.y + dify) in self.labyrinth.river:
                if (self.bear.x, self.bear.y) not in self.labyrinth.river:
                    idx = self.labyrinth.river.index((self.bear.x + difx, self.bear.y + dify))
                    
                    if idx + 2 < len(self.labyrinth.river):
                        self.bear.x, self.bear.y = self.labyrinth.river[idx + 2]
                    else:
                        self.bear.x, self.bear.y = self.labyrinth.river[-1]
                else:
                    self.bear.move(direction)
            
            else:
                self.bear.move(direction)
            
            
        if (self.bear.x, self.bear.y) == (self.player.x, self.player.y):
            self.bear.hit(self.player)
            
            walls = set(['up', 'down', 'right', 'left'])
            directions = list(walls.difference(self.labyrinth.cells[self.bear.x][self.bear.y].walls))
            action = random.choice(directions)
            command = 'go ' + action

            print('You got bited by bear!')
            print(f'Your current hp is: {self.player.hp}')
            
            if self.player.hp <= 0:
                return False
            
            print()
            print(f'You pushed into direction: {action}')
            self.move(command)
        
        return True
    
    
    def get_hp(self, command):
        '''Get current hp of player'''
        print(f'Your current hp is: {self.player.hp}')
        
        return None
        
            
    def create_newgame(self):
        initial_cell_player = self.labyrinth.get_random_cell()
        initial_cell_bear = self.labyrinth.get_random_cell()
        
        while initial_cell_player == initial_cell_bear:
            initial_cell_bear = self.labyrinth.get_random_cell()
        
        self.player = Player(initial_cell_player.x, initial_cell_player.y)
        self.bear = Bear(initial_cell_bear.x, initial_cell_bear.y)
        
        self.labyrinth.cells[self.player.x][self.player.y].explored()
        self.__won = False
      
    
    def play(self, loaded=False, debug_mode=False):
           
        if not loaded:
            self.create_newgame()
        
        while not self.__won:
            try:
                if debug_mode:
                    self.clear(None)
                    self.print_labyrinth(None)
                    
                command = input()
                result = self.commands[command.split()[0]](command)
                
                # No step done, bear don't move
                if result is None:
                    continue
                
                # Quit the game
                if result is False:
                    return False
                
                result = self.bear_walking()
                
                # Killed by bear
                if result is False:
                    print('You died')
                    return False
            
            except (KeyError, IndexError):
                print('Unknown command')
                
        print('You won!')
        return True

In [114]:
class Menu(ABC):
    
    @abstractmethod
    def new_game(self):
        pass
    
    
    @abstractmethod
    def load_game(self):
        pass

In [115]:
class LabyrinthMenu(Menu):
    
    
    def start(self):
        print('Type command "start <labyrinth_size>", "load <file_name>" or "quit"')
        
        while True:
            
            self.command = input().split()
            if not self.command:
                continue
                
            if self.command[0] == 'start':
                self.command[1] = int(self.command[1])
                if (self.command[1] < 4) or (self.command[1] > 10):
                    print('Size must be not less 4 and not bigger 10')
                    continue
                return self.new_game()
            
            elif self.command[0] == 'debug':
                self.command[1] = int(self.command[1])
                if (self.command[1] < 4) or (self.command[1] > 10):
                    print('Size must be not less 4 and not bigger 10')
                    continue
                return self.new_game(debug_mode=True)
            
            elif self.command[0] == 'load':
                self.command[1] = self.command[1] + '.pkl'
                return self.load_game()
            
            elif self.command[0] == 'quit':
                return False
            
            else:
                print('Unknown command, try again or type "quit"')
    
    
    def new_game(self, debug_mode=False):
        fabric = LabyrinthFabric()
        new_game = LabyrinthGame(fabric.produce(self.command[1]))
        print('New game has started')
        
        return new_game.play(debug_mode=debug_mode)
    
    
    def load_game(self):
        with open(os.path.join('saves', self.command[1]), 'rb') as f:
            loaded_game = pickle.load(f)
            
        print('Loaded game has started')
        
        return loaded_game.play(loaded=True)

In [77]:
if __name__ == '__main__':
    LabyrinthMenu().start()

Type command "start <labyrinth_size>", "load <file_name>" or "quit"
quit


In [125]:
lab = Labyrinth(8).create_labyrinth()
print_labyrinth(lab)

# ########
#┌─$╶┬┐╶┐#
#│┌┘┌┘└┐@#
#││┌┘┌┐└┤#
#│└┘╶@└┬┘#
#│┌┐@─┐@┐#
#├┘│@╴│≈≈#
#│╷└┘┌┘╷≈#
#└┴─╴└─┘*#
##########


In [37]:
def print_labyrinth(self):
    for i in range(len(self.map)):
        for j in range(len(self.map)):
            print(self.map[j][i], end='')
        print()

In [93]:
set([1,2]).diffdifference([2])

{1}